# Things that need preparing
## CHANGE_SSL_METHOD
### change_for_archiecture

* old resnet
* new resnet

****

### SSL methods used

* Barlow_twins
* simsiam
* supcon
* swav

****

## Domain adaptation 
* MNISIT
* MNISIT_MIX


In [1]:
import wandb

from utils.util import BCE, PairEnum, cluster_acc, Identity, AverageMeter, seed_torch,accuracy
from data.cifarloader import CIFAR10Loader, CIFAR10LoaderMix, CIFAR100Loader, CIFAR100LoaderMix
from models.resnet import ResNet, BasicBlock
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from sklearn.metrics import adjusted_rand_score as ari_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, lr_scheduler
import numpy as np
from tqdm import tqdm
import pandas as pd


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# all these conditions for cifar 10 
seed_torch(0)
num_labeled_classes=5
num_unlabeled_classes=5
num_classes = num_labeled_classes + num_unlabeled_classes
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [5]:
api = wandb.Api()# used to import weight and bias pipeline


## STEP 3
## Change of architecutre

In [6]:
## STEP 1
run_old_SSL = api.run("mhaggag96/trends_project/1ni39qdj").history()
run_new_SSL = api.run("mhaggag96/trends_project/1o6drm9k").history()
run_old_SSL_model = 'data/experiments/selfsupervised_learning/rotnet_cifar10_basicconfig.pth'
run_new_SSL_model = 'data/experiments/selfsupervised_learning/rotnet_cifar10_new_config.pth'
## STEP 2
run_old_SL = api.run("mhaggag96/trends_project/vp78kjp8").history()
run_new_SL = api.run("mhaggag96/trends_project/2sderoky").history()
run_old_SL_model = 'data/experiments/supervised_learning/resnet_rotnet_cifar10_basicconfig.pth'
run_new_SL_model = 'data/experiments/supervised_learning/resnet_rotnet_cifar10_new_config.pth'
## STEP 3
run_old_NCL = api.run("mhaggag96/trends_project/1ggpmba1").history()
run_new_NCL = api.run("mhaggag96/trends_project/2wov4ow5").history()
run_old_SSL_model = 'data/experiments/auto_novel/resnet_IL_cifar10_basic_config.pth'
run_old_SSL_model = 'data/experiments/auto_novel/resnet_IL_cifar10_new_config.pth'

### analysis on SSL step 

In [7]:
from selfsupervised_learning import resnet_sim,ResNet


In [8]:
model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=4)


NameError: name 'is_adapters' is not defined